<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_sarima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

! mkdir ~/.kaggle
from google.colab import files
files.upload()
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/

!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting
!unzip '*.csv.zip'
!unzip '*.csv.zip'


Mounted at /content/drive


Saving kaggle.json to kaggle.json
total 4
-rw------- 1 root root 71 Jul  6 17:30 kaggle.json
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 851MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  test.csv.zip
  inflating: test.csv                

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
  inflating: train.csv               

4 archives were successfully processed.
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zi

In [2]:
!pip install -q dagshub mlflow


import pandas as pd
import numpy as np
import gc
import mlflow
import dagshub

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")


dagshub.init(repo_owner='eghib22', repo_name='Store-Sales-Forecasting', mlflow=True)
mlflow.set_experiment("SARIMAX_Modeling")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=998a8b5c-d2d5-43d2-8bbc-35eb89ae4004&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=9380c1de0468e0544054bb806819946c4ca59e4bb839c2370cc1368981c6e5d9




Accessing as agasi22

Initialized MLflow to track repo "eghib22/Store-Sales-Forecasting"

Repository eghib22/Store-Sales-Forecasting initialized!

2025/07/06 17:31:15 INFO mlflow.tracking.fluent: Experiment with name 'SARIMAX_Modeling' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/0a62617f296e457a99d3d8715c61e659', creation_time=1751823075278, experiment_id='1', last_update_time=1751823075278, lifecycle_stage='active', name='SARIMAX_Modeling', tags={}>

In [3]:
train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])


In [4]:

df = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
df = df.merge(stores, on='Store', how='left')

exog_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
df[exog_cols] = df[exog_cols].fillna(method='ffill').fillna(method='bfill')

df['Month'] = df['Date'].dt.month
df['WeekOfYear'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.year

df['HolidayWeek'] = df.groupby(['Store', 'Dept'])['IsHoliday'].transform('max')

exog_cols.extend(['Month', 'WeekOfYear', 'HolidayWeek'])
print(f"Using exogenous features: {exog_cols}")


Using exogenous features: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'WeekOfYear', 'HolidayWeek']


In [5]:
def weighted_mae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [8]:
X = g[exog_cols].copy()
# Force all exog columns numeric
X = X.apply(pd.to_numeric, errors='coerce')
# Fill any remaining NaNs
X = X.fillna(method='ffill').fillna(method='bfill').fillna(0)

# Drop any NaNs in y just in case
y = pd.to_numeric(y, errors='coerce').fillna(0)


In [11]:
print(">>> Checking exog columns types:")
print(df[exog_cols].dtypes)
print(">>> Checking Weekly_Sales type:")
print(df['Weekly_Sales'].dtype)
print(">>> Checking for any NaNs:")
print(df[exog_cols].isnull().sum())
print(df['Weekly_Sales'].isnull().sum())


>>> Checking exog columns types:
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
Month             int32
WeekOfYear       UInt32
HolidayWeek        bool
dtype: object
>>> Checking Weekly_Sales type:
float64
>>> Checking for any NaNs:
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
Month           0
WeekOfYear      0
HolidayWeek     0
dtype: int64
0


In [13]:
results = []
all_preds = []

store_dept_groups = df.groupby(['Store', 'Dept'])
total_groups = len(store_dept_groups)

print(f"--- Starting SARIMAX for {total_groups} Store-Dept combos ---")

with mlflow.start_run(run_name="SARIMAX_Forecasting"):
    mlflow.log_param("SARIMAX_order", "(1,1,1)")
    mlflow.log_param("SARIMAX_seasonal_order", "(1,1,1,52)")
    mlflow.log_param("exogenous_features", exog_cols)

    for idx, ((store_id, dept_id), group) in enumerate(store_dept_groups, start=1):
        print(f"\n--- Processing Store: {store_id}, Dept: {dept_id} ({idx}/{total_groups}) ---")

        g = group.sort_values('Date')
        g = g.set_index('Date')

        y = g['Weekly_Sales']
        X = g[exog_cols].copy()
        weights = g['IsHoliday'].apply(lambda x: 5 if x else 1)

        y_train = y[y.index < '2012-01-01']
        y_val = y[(y.index >= '2012-01-01') & (y.index < '2012-07-01')]
        X_train = X.loc[y_train.index].copy()
        X_val = X.loc[y_val.index].copy()
        weights_val = weights.loc[y_val.index]

        if len(y_train) < 100 or len(y_val) < 20:
            print(f"   Skipped: Not enough data (Train: {len(y_train)}, Val: {len(y_val)})")
            continue

        try:
            # 👉 Fix bool exog to int
            if 'HolidayWeek' in X_train.columns:
                X_train['HolidayWeek'] = X_train['HolidayWeek'].astype(int)
                X_val['HolidayWeek'] = X_val['HolidayWeek'].astype(int)

            # Ensure all exog are float64
            X_train = X_train.astype('float64')
            X_val = X_val.astype('float64')
            y_train = y_train.astype('float64')
            y_val = y_val.astype('float64')

            # Check index types
            y_train.index = pd.to_datetime(y_train.index)
            X_train.index = pd.to_datetime(X_train.index)

            model = SARIMAX(
                y_train,
                exog=X_train,
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 52),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            result = model.fit(disp=False)

            pred = result.predict(
                start=y_val.index[0],
                end=y_val.index[-1],
                exog=X_val
            )

            wmae = weighted_mae(y_val, pred, weights_val)
            rmse = np.sqrt(mean_squared_error(y_val, pred))

            print(f"   WMAE: {wmae:.2f}")

            results.append({
                'Store': store_id,
                'Dept': dept_id,
                'RMSE': rmse,
                'WMAE': wmae
            })

            all_preds.append(pd.DataFrame({
                'y_true': y_val.values,
                'y_pred': pred.values,
                'weight': weights_val.values
            }))

        except Exception as e:
            print(f"   Failed: {e}")
            continue

    print("\n--- SARIMAX Loop Finished ---")

    if len(all_preds) > 0:
        all_df = pd.concat(all_preds)
        overall_wmae = np.sum(all_df['weight'] * np.abs(all_df['y_true'] - all_df['y_pred'])) / np.sum(all_df['weight'])
        print(f"Overall WMAE: {overall_wmae:.2f}")
        mlflow.log_metric('Overall_WMAE', overall_wmae)
    else:
        print("No valid predictions generated, skipping overall WMAE calculation.")

    mlflow.log_metric('total_groups_processed', len(results))


Streaming output truncated to the last 5000 lines.

--- Processing Store: 22, Dept: 78 (1667/3331) ---
   Skipped: Not enough data (Train: 6, Val: 0)

--- Processing Store: 22, Dept: 79 (1668/3331) ---
   WMAE: 2093.77

--- Processing Store: 22, Dept: 80 (1669/3331) ---
   WMAE: 274.28

--- Processing Store: 22, Dept: 81 (1670/3331) ---
   WMAE: 1398.94

--- Processing Store: 22, Dept: 82 (1671/3331) ---
   WMAE: 6545.04

--- Processing Store: 22, Dept: 83 (1672/3331) ---
   WMAE: 327.27

--- Processing Store: 22, Dept: 85 (1673/3331) ---
   WMAE: 1714.33

--- Processing Store: 22, Dept: 87 (1674/3331) ---
   WMAE: 2737.80

--- Processing Store: 22, Dept: 90 (1675/3331) ---
   WMAE: 1409.23

--- Processing Store: 22, Dept: 91 (1676/3331) ---
   WMAE: 2535.18

--- Processing Store: 22, Dept: 92 (1677/3331) ---
   WMAE: 3354.67

--- Processing Store: 22, Dept: 93 (1678/3331) ---
   WMAE: 417.40

--- Processing Store: 22, Dept: 94 (1679/3331) ---
   Skipped: Not enough data (Train: 81, Va

In [14]:
results_df = pd.DataFrame(results)
print(results_df.head())

mlflow.end_run()


   Store  Dept          RMSE          WMAE
0      1     1  24963.706264  23829.055052
1      1     2   5720.070163   5227.739413
2      1     3   1492.652893   1132.798734
3      1     4   5552.193749   5018.970811
4      1     5  14432.069454  12949.813600
